In [34]:
from ultralytics import YOLO
YOLO("../models/yolo11s.pt")
# YOLO("../models/yolo11m.pt")
# YOLO("../models/yolo11l.pt")
# YOLO("../models/yolo11x.pt")


YOLO(
  (model): DetectionModel(
    (model): Sequential(
      (0): Conv(
        (conv): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(32, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
      (1): Conv(
        (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(64, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
      (2): C3k2(
        (cv1): Conv(
          (conv): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn): BatchNorm2d(64, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
          (act): SiLU(inplace=True)
        )
        (cv2): Conv(
          (conv): Conv2d(96, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn): BatchNorm2d(128, eps=0.001, momentum=0.03, affine=True, track_runnin

In [35]:
import torch

In [36]:
import torch
print(torch.cuda.is_available())

True


In [37]:
print(torch.cuda.get_device_name(0))
print(torch.cuda.current_device())
print(torch.cuda.memory_allocated())
print(torch.cuda.memory_reserved())


NVIDIA GeForce RTX 2060
0
0
0


In [38]:
import os
import cv2
import torch
import pandas as pd
from datetime import datetime, timedelta
from imutils.video import FileVideoStream
from ultralytics import YOLO

In [39]:
video_dir = "../data/raw/2025-11-05"  # folder with all hourly videos
frame_dir = "../data/frames/cafe_pos_person_2025_11_05"
model_path = "../models/yolo11s.pt"
csv_path = "../db/cafe_pos_2025-11-05.csv"  # your exported Accounting DB CSV

confidence_threshold = 0.8
frame_skip = 1 #2
time_window = timedelta(minutes=1.75)  # 2-minute window after sale time

os.makedirs(frame_dir, exist_ok=True)

In [40]:
df = pd.read_csv(csv_path)
df["AccoDate"] = pd.to_datetime(df["AccoDateStamp"])
unique_txns = df.groupby("AccoID")["AccoDate"].min().reset_index()
print(f"🧾 Loaded {len(unique_txns)} unique accounting transactions")

🧾 Loaded 199 unique accounting transactions


In [41]:
df

,AccoID,AccoDate,AccoDocNo,AccoTypeId,AccoAmount,AccoDebit,AccoCredit,VatRateId,VatAmount,SessionID,...,WeekNo,Total,AccoDateStamp,Quantity,Discount,UnitPrice,StockCateDesc,StockName,StockSizeDesc,StockDesciption
0,1168504,2025-11-05 08:20:18.357,224522/161,7,-103.2,103.2,0.0,0,0.0,19303,...,45,True,2025-11-05 08:20:18.357,4.0,0.0,8.0,Extra,EXTRA,NaN,egg breakfast
1,1168504,2025-11-05 08:20:18.357,224522/161,7,-103.2,103.2,0.0,0,0.0,19303,...,45,True,2025-11-05 08:20:18.357,1.0,0.2,26.0,Beverages Tea,TEA,NaN,earl grey
2,1168504,2025-11-05 08:20:18.357,224522/161,7,-103.2,103.2,0.0,0,0.0,19303,...,45,True,2025-11-05 08:20:18.357,1.0,0.2,63.0,Breakfast,BREAKFAST,NaN,granola
3,1168509,2025-11-05 08:29:49.110,224523/1,7,-72.0,72.0,0.0,0,0.0,19305,...,45,True,2025-11-05 08:29:49.110,2.0,0.0,36.0,Beverages Coffee,COFFEE,large,cappuccino
4,1168514,2025-11-05 08:33:00.143,224524,7,-264.0,264.0,0.0,0,0.0,19305,...,45,True,2025-11-05 08:33:00.143,2.0,0.0,24.0,Beverages Tea,TEA,NaN,rooibos
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
532,1169788,2025-11-05 16:21:31.267,224720/198,7,-237.0,237.0,0.0,0,0.0,19309,...,45,True,2025-11-05 16:21:31.267,1.0,0.0,59.0,Cakes,CARROT CAKE,Slice,carrot cake
533,1169788,2025-11-05 16:21:31.267,224720/198,7,-237.0,237.0,0.0,0,0.0,19309,...,45,True,2025-11-05 16:21:31.267,2.0,0.0,89.0,Cakes Cheese,CHEESE CAKE,NaN,milo
534,1169797,2025-11-05 16:30:48.420,224722,7,-78.0,78.0,0.0,0,0.0,19309,...,45,True,2025-11-05 16:30:48.420,1.0,0.0,29.0,Mineral cooldrinks,COKE,300ml,zero
535,1169797,2025-11-05 16:30:48.420,224722,7,-78.0,78.0,0.0,0,0.0,19309,...,45,True,2025-11-05 16:30:48.420,1.0,0.0,29.0,Mineral cooldrinks,CREAM SODA,340ml,can


In [42]:
device = "cuda" if torch.cuda.is_available() else "cpu"
model = YOLO(model_path)
print(f"🚀 Model loaded on {device.upper()}")

🚀 Model loaded on CUDA


In [43]:
def get_matching_txns(frame_time):
    mask = (unique_txns["AccoDate"] >= frame_time) & \
           (unique_txns["AccoDate"] <= frame_time + time_window)
    return unique_txns.loc[mask, "AccoID"].tolist()

In [44]:
def process_video(video_path, start_time):
    print(f"\n▶️ Processing video: {os.path.basename(video_path)}")
    fvs = FileVideoStream(video_path).start()
    torch.cuda.empty_cache()
    frame_id, saved = 0, 0
    fps = 25  # adjust to match your camera if needed

    while fvs.more():
        frame = fvs.read()
        if frame is None:
            break

        # Frame skipping
        if frame_id % frame_skip != 0:
            frame_id += 1
            continue

        # Compute current frame timestamp
        elapsed = frame_id / fps
        current_time = start_time + timedelta(seconds=elapsed)

        # Find matching transactions
        matching_ids = get_matching_txns(current_time)
        if not matching_ids:
            frame_id += 1
            continue

        # YOLO detection
        results = model(frame, device=device, conf=confidence_threshold, verbose=False)[0]
        boxes = results.boxes

        if boxes is not None and len(boxes) > 0:
            frame_h, frame_w = frame.shape[:2]
            classes = boxes.cls.cpu().numpy().astype(int)
            coords = boxes.xyxy.cpu().numpy().astype(int)
            confs = boxes.conf.cpu().numpy()

            # Check for close-up people
            close_person_found = False
            for cls_id, (x1, y1, x2, y2), conf in zip(classes, coords, confs):
                if cls_id != 0 or conf < confidence_threshold:
                    continue
                box_w, box_h = x2 - x1, y2 - y1
                if (box_h / frame_h) >= 0.3 and (box_w / frame_w) >= 0.2:
                    close_person_found = True
                    break

            # Save frame if a close-up person is found
            if close_person_found:
                for acco_id in matching_ids:
                    timestamp_str = current_time.strftime("%Y%m%d_%H%M%S")
                    out_name = f"AccoID_{acco_id}_{timestamp_str}.jpg"
                    out_path = os.path.join(frame_dir, out_name)
                    cv2.imwrite(out_path, frame)
                    saved += 1

        frame_id += 1

    fvs.stop()
    print(f"✅ Finished {os.path.basename(video_path)} — {saved} frames saved.")


In [45]:
# MAIN LOOP: Process all videos for the day
# ──────────────────────────────
videos = sorted([
    os.path.join(video_dir, f)
    for f in os.listdir(video_dir)
    if f.lower().endswith(".mp4")
])

# Define start times for each video
day_date = datetime(2025, 11, 5, 7, 59, 58)  # first clip starts 08:30
clip_length = timedelta(hours=1)          # each video = 1 hour

for i, video_path in enumerate(videos):
    start_time = day_date + i * clip_length
    process_video(video_path, start_time)

print("\n🎯 All videos processed successfully!")


▶️ Processing video: CafePOS_20251105_0800_to_0900.mp4
✅ Finished CafePOS_20251105_0800_to_0900.mp4 — 12507 frames saved.

▶️ Processing video: CafePOS_20251105_0900_to_1000.mp4
✅ Finished CafePOS_20251105_0900_to_1000.mp4 — 65524 frames saved.

▶️ Processing video: CafePOS_20251105_1000_to_1100.mp4
✅ Finished CafePOS_20251105_1000_to_1100.mp4 — 59342 frames saved.

▶️ Processing video: CafePOS_20251105_1100_to_1200.mp4
✅ Finished CafePOS_20251105_1100_to_1200.mp4 — 62966 frames saved.

▶️ Processing video: CafePOS_20251105_1200_to_1300.mp4
✅ Finished CafePOS_20251105_1200_to_1300.mp4 — 78032 frames saved.

▶️ Processing video: CafePOS_20251105_1300_to_1400.mp4
✅ Finished CafePOS_20251105_1300_to_1400.mp4 — 51101 frames saved.

▶️ Processing video: CafePOS_20251105_1400_to_1500.mp4
✅ Finished CafePOS_20251105_1400_to_1500.mp4 — 23131 frames saved.

▶️ Processing video: CafePOS_20251105_1500_to_1600.mp4
✅ Finished CafePOS_20251105_1500_to_1600.mp4 — 9984 frames saved.

▶️ Processing vi

In [46]:
import os
import cv2
from ultralytics import YOLO

# ──────────────────────────────
# PATHS
# ──────────────────────────────
input_dir = "../data/frames/cafe_pos_person_2025_11_05"
output_dir = "../data/crops_person/cafe_POS_person_crops_2025_11_05"
os.makedirs(output_dir, exist_ok=True)

# ──────────────────────────────
# YOLO MODEL SETUP
# ──────────────────────────────
model = YOLO("../models/yolo11s.pt")  # or your YOLO version
CONFIDENCE_THRESHOLD = 0.8

# Minimum % of frame height for a person to be considered “close-up”
MIN_HEIGHT_RATIO = 0.30   # 0.35 → person must occupy at least 35% of the frame height
MIN_WIDTH_RATIO = 0.15   # 0.20 → optional width threshold

# ──────────────────────────────
# PROCESS FRAMES
# ──────────────────────────────
images = sorted([f for f in os.listdir(input_dir) if f.lower().endswith((".jpg", ".png"))])
print(f"🧩 Found {len(images)} frames to process.\n")

for i, img_name in enumerate(images):
    img_path = os.path.join(input_dir, img_name)
    frame = cv2.imread(img_path)
    if frame is None:
        print(f"⚠️ Skipped unreadable image: {img_name}")
        continue

    frame_h, frame_w = frame.shape[:2]

    # YOLO detection
    results = model(frame, verbose=False)[0]
    boxes = results.boxes
    if boxes is None or len(boxes) == 0:
        continue

    # Extract detections
    class_ids = boxes.cls.cpu().numpy().astype(int)
    coords = boxes.xyxy.cpu().numpy().astype(int)
    confs = boxes.conf.cpu().numpy()

    person_found = False

    for j, (cls_id, (x1, y1, x2, y2), conf) in enumerate(zip(class_ids, coords, confs)):
        if cls_id != 0 or conf < CONFIDENCE_THRESHOLD:
            continue  # keep only 'person'

        # Compute relative size of bounding box
        box_w = x2 - x1
        box_h = y2 - y1
        height_ratio = box_h / frame_h
        width_ratio = box_w / frame_w

        # Skip small (far-away) people
        if height_ratio < MIN_HEIGHT_RATIO or width_ratio < MIN_WIDTH_RATIO:
            continue

        crop = frame[y1:y2, x1:x2]

        # Skip any tiny crops that remain (failsafe)
        if crop.shape[0] < 80 or crop.shape[1] < 80:
            continue

        # Save crop with same traceable frame name
        base_name, ext = os.path.splitext(img_name)
        out_name = f"{base_name}{j}{ext}"
        out_path = os.path.join(output_dir, out_name)
        cv2.imwrite(out_path, crop, [int(cv2.IMWRITE_JPEG_QUALITY), 95])
        person_found = True

    if person_found:
        print(f"✅ Saved close-up crop(s) from: {img_name}")

print(f"\n🎯 Done! Close-up person crops saved to: {output_dir}")


🧩 Found 15546 frames to process.

✅ Saved close-up crop(s) from: AccoID_1168504_20251105_081836.jpg
✅ Saved close-up crop(s) from: AccoID_1168504_20251105_081842.jpg
✅ Saved close-up crop(s) from: AccoID_1168504_20251105_081843.jpg
✅ Saved close-up crop(s) from: AccoID_1168504_20251105_081858.jpg
✅ Saved close-up crop(s) from: AccoID_1168504_20251105_081906.jpg
✅ Saved close-up crop(s) from: AccoID_1168504_20251105_081939.jpg
✅ Saved close-up crop(s) from: AccoID_1168504_20251105_081940.jpg
✅ Saved close-up crop(s) from: AccoID_1168504_20251105_081941.jpg
✅ Saved close-up crop(s) from: AccoID_1168504_20251105_081942.jpg
✅ Saved close-up crop(s) from: AccoID_1168504_20251105_081943.jpg
✅ Saved close-up crop(s) from: AccoID_1168504_20251105_081944.jpg
✅ Saved close-up crop(s) from: AccoID_1168504_20251105_081945.jpg
✅ Saved close-up crop(s) from: AccoID_1168504_20251105_081946.jpg
✅ Saved close-up crop(s) from: AccoID_1168504_20251105_081947.jpg
✅ Saved close-up crop(s) from: AccoID_1168

In [47]:
video_dir = "../data/raw/2025-11-07"  # folder with all hourly videos
frame_dir = "../data/frames/cafe_pos_person_2025_11_07"
model_path = "../models/yolo11s.pt"
csv_path = "../db/cafe_pos_2025-11-07.csv"  # your exported Accounting DB CSV

confidence_threshold = 0.8
frame_skip = 1 #2
time_window = timedelta(minutes=1.75)  # 2-minute window after sale time

os.makedirs(frame_dir, exist_ok=True)

In [48]:
df = pd.read_csv(csv_path)
df["AccoDate"] = pd.to_datetime(df["AccoDateStamp"])
unique_txns = df.groupby("AccoID")["AccoDate"].min().reset_index()
print(f"🧾 Loaded {len(unique_txns)} unique accounting transactions")

🧾 Loaded 189 unique accounting transactions


In [49]:
device = "cuda" if torch.cuda.is_available() else "cpu"
model = YOLO(model_path)
print(f"🚀 Model loaded on {device.upper()}")

🚀 Model loaded on CUDA


In [50]:
def get_matching_txns(frame_time):
    mask = (unique_txns["AccoDate"] >= frame_time) & \
           (unique_txns["AccoDate"] <= frame_time + time_window)
    return unique_txns.loc[mask, "AccoID"].tolist()

In [51]:
def process_video(video_path, start_time):
    print(f"\n▶️ Processing video: {os.path.basename(video_path)}")
    fvs = FileVideoStream(video_path).start()
    torch.cuda.empty_cache()
    frame_id, saved = 0, 0
    fps = 25  # adjust to match your camera if needed

    while fvs.more():
        frame = fvs.read()
        if frame is None:
            break

        # Frame skipping
        if frame_id % frame_skip != 0:
            frame_id += 1
            continue

        # Compute current frame timestamp
        elapsed = frame_id / fps
        current_time = start_time + timedelta(seconds=elapsed)

        # Find matching transactions
        matching_ids = get_matching_txns(current_time)
        if not matching_ids:
            frame_id += 1
            continue

        # YOLO detection
        results = model(frame, device=device, conf=confidence_threshold, verbose=False)[0]
        boxes = results.boxes

        if boxes is not None and len(boxes) > 0:
            frame_h, frame_w = frame.shape[:2]
            classes = boxes.cls.cpu().numpy().astype(int)
            coords = boxes.xyxy.cpu().numpy().astype(int)
            confs = boxes.conf.cpu().numpy()

            # Check for close-up people
            close_person_found = False
            for cls_id, (x1, y1, x2, y2), conf in zip(classes, coords, confs):
                if cls_id != 0 or conf < confidence_threshold:
                    continue
                box_w, box_h = x2 - x1, y2 - y1
                if (box_h / frame_h) >= 0.3 and (box_w / frame_w) >= 0.2:
                    close_person_found = True
                    break

            # Save frame if a close-up person is found
            if close_person_found:
                for acco_id in matching_ids:
                    timestamp_str = current_time.strftime("%Y%m%d_%H%M%S")
                    out_name = f"AccoID_{acco_id}_{timestamp_str}.jpg"
                    out_path = os.path.join(frame_dir, out_name)
                    cv2.imwrite(out_path, frame)
                    saved += 1

        frame_id += 1

    fvs.stop()
    print(f"✅ Finished {os.path.basename(video_path)} — {saved} frames saved.")


In [52]:
# MAIN LOOP: Process all videos for the day
# ──────────────────────────────
videos = sorted([
    os.path.join(video_dir, f)
    for f in os.listdir(video_dir)
    if f.lower().endswith(".mp4")
])

# Define start times for each video
day_date = datetime(2025, 11, 7, 7, 59, 58)  # first clip starts 08:30
clip_length = timedelta(hours=1)          # each video = 1 hour

for i, video_path in enumerate(videos):
    start_time = day_date + i * clip_length
    process_video(video_path, start_time)

print("\n🎯 All videos processed successfully!")


▶️ Processing video: CafePOS_20251107_0800_to_0900.mp4
✅ Finished CafePOS_20251107_0800_to_0900.mp4 — 7179 frames saved.

▶️ Processing video: CafePOS_20251107_0900_to_1000.mp4
✅ Finished CafePOS_20251107_0900_to_1000.mp4 — 60861 frames saved.

▶️ Processing video: CafePOS_20251107_1000_to_1100.mp4
✅ Finished CafePOS_20251107_1000_to_1100.mp4 — 76681 frames saved.

▶️ Processing video: CafePOS_20251107_1100_to_1200.mp4
✅ Finished CafePOS_20251107_1100_to_1200.mp4 — 34476 frames saved.

▶️ Processing video: CafePOS_20251107_1200_to_1300.mp4
✅ Finished CafePOS_20251107_1200_to_1300.mp4 — 41930 frames saved.

▶️ Processing video: CafePOS_20251107_1300_to_1400.mp4
✅ Finished CafePOS_20251107_1300_to_1400.mp4 — 69842 frames saved.

▶️ Processing video: CafePOS_20251107_1400_to_1500.mp4
✅ Finished CafePOS_20251107_1400_to_1500.mp4 — 64464 frames saved.

▶️ Processing video: CafePOS_20251107_1500_to_1600.mp4
✅ Finished CafePOS_20251107_1500_to_1600.mp4 — 9967 frames saved.

▶️ Processing vid

In [53]:
import os
import cv2
from ultralytics import YOLO

# ──────────────────────────────
# PATHS
# ──────────────────────────────
input_dir = "../data/frames/cafe_pos_person_2025_11_07"
output_dir = "../data/crops_person/cafe_POS_person_crops_2025_11_07"
os.makedirs(output_dir, exist_ok=True)

# ──────────────────────────────
# YOLO MODEL SETUP
# ──────────────────────────────
model = YOLO("../models/yolo11s.pt")  # or your YOLO version
CONFIDENCE_THRESHOLD = 0.8

# Minimum % of frame height for a person to be considered “close-up”
MIN_HEIGHT_RATIO = 0.30   # 0.35 → person must occupy at least 35% of the frame height
MIN_WIDTH_RATIO = 0.15   # 0.20 → optional width threshold

# ──────────────────────────────
# PROCESS FRAMES
# ──────────────────────────────
images = sorted([f for f in os.listdir(input_dir) if f.lower().endswith((".jpg", ".png"))])
print(f"🧩 Found {len(images)} frames to process.\n")

for i, img_name in enumerate(images):
    img_path = os.path.join(input_dir, img_name)
    frame = cv2.imread(img_path)
    if frame is None:
        print(f"⚠️ Skipped unreadable image: {img_name}")
        continue

    frame_h, frame_w = frame.shape[:2]

    # YOLO detection
    results = model(frame, verbose=False)[0]
    boxes = results.boxes
    if boxes is None or len(boxes) == 0:
        continue

    # Extract detections
    class_ids = boxes.cls.cpu().numpy().astype(int)
    coords = boxes.xyxy.cpu().numpy().astype(int)
    confs = boxes.conf.cpu().numpy()

    person_found = False

    for j, (cls_id, (x1, y1, x2, y2), conf) in enumerate(zip(class_ids, coords, confs)):
        if cls_id != 0 or conf < CONFIDENCE_THRESHOLD:
            continue  # keep only 'person'

        # Compute relative size of bounding box
        box_w = x2 - x1
        box_h = y2 - y1
        height_ratio = box_h / frame_h
        width_ratio = box_w / frame_w

        # Skip small (far-away) people
        if height_ratio < MIN_HEIGHT_RATIO or width_ratio < MIN_WIDTH_RATIO:
            continue

        crop = frame[y1:y2, x1:x2]

        # Skip any tiny crops that remain (failsafe)
        if crop.shape[0] < 80 or crop.shape[1] < 80:
            continue

        # Save crop with same traceable frame name
        base_name, ext = os.path.splitext(img_name)
        out_name = f"{base_name}{j}{ext}"
        out_path = os.path.join(output_dir, out_name)
        cv2.imwrite(out_path, crop, [int(cv2.IMWRITE_JPEG_QUALITY), 95])
        person_found = True

    if person_found:
        print(f"✅ Saved close-up crop(s) from: {img_name}")

print(f"\n🎯 Done! Close-up person crops saved to: {output_dir}")


🧩 Found 15347 frames to process.

✅ Saved close-up crop(s) from: AccoID_1170787_20251107_083423.jpg
✅ Saved close-up crop(s) from: AccoID_1170787_20251107_083424.jpg
✅ Saved close-up crop(s) from: AccoID_1170787_20251107_083425.jpg
✅ Saved close-up crop(s) from: AccoID_1170787_20251107_083426.jpg
✅ Saved close-up crop(s) from: AccoID_1170787_20251107_083427.jpg
✅ Saved close-up crop(s) from: AccoID_1170787_20251107_083428.jpg
✅ Saved close-up crop(s) from: AccoID_1170787_20251107_083429.jpg
✅ Saved close-up crop(s) from: AccoID_1170787_20251107_083430.jpg
✅ Saved close-up crop(s) from: AccoID_1170787_20251107_083431.jpg
✅ Saved close-up crop(s) from: AccoID_1170787_20251107_083432.jpg
✅ Saved close-up crop(s) from: AccoID_1170787_20251107_083433.jpg
✅ Saved close-up crop(s) from: AccoID_1170787_20251107_083434.jpg
✅ Saved close-up crop(s) from: AccoID_1170787_20251107_083435.jpg
✅ Saved close-up crop(s) from: AccoID_1170787_20251107_083436.jpg
✅ Saved close-up crop(s) from: AccoID_1170